   # Design of Item-Based Collaborative Filtering Model

## Merge the datasets Development.csv and IT_Software.csv

#### Import Modules

In [1]:
# Import Modules

import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn import preprocessing

import re, math
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.corpus import wordnet as wn
import ast
import random

%matplotlib inline

#### df_dev Preprocessing

In [2]:
df_dev = pd.read_csv("Dataset/Development.csv")

In [3]:
df_dev.head()

,id,title,url,is_paid,num_subscribers,avg_rating,avg_rating_recent,rating,num_reviews,num_published_lectures,num_published_practice_tests,created,published_time,discount_price__amount,discount_price__currency,price_detail__amount,price_detail__currency
0,567828,2020 Complete Python Bootcamp: From Zero to He...,/course/complete-python-bootcamp/,True,1086954,4.56326,4.56743,4.56743,314098,152,0,2015-07-29T00:12:23Z,2015-10-12T21:42:53Z,455.0,INR,8640,INR
1,625204,The Web Developer Bootcamp,/course/the-web-developer-bootcamp/,True,592474,4.64777,4.65291,4.65291,181621,405,0,2015-09-28T21:32:19Z,2015-11-02T21:13:27Z,455.0,INR,8640,INR
2,950390,Machine Learning A-Z™: Hands-On Python & R In ...,/course/machinelearning/,True,687330,4.55210,4.54932,4.54932,131007,323,0,2016-09-05T09:54:22Z,2016-09-13T21:43:44Z,455.0,INR,8640,INR
3,756150,Angular - The Complete Guide (2020 Edition),/course/the-complete-guide-to-angular-2/,True,417746,4.57687,4.58594,4.58594,128705,455,0,2016-02-08T17:02:55Z,2016-02-11T07:29:29Z,462.0,INR,8640,INR
4,533682,Java Programming Masterclass for Software Deve...,/course/java-the-complete-java-developer-course/,True,497195,4.56571,4.57203,4.57203,122306,400,0,2015-06-21T20:25:56Z,2015-07-21T22:01:13Z,455.0,INR,8640,INR


In [4]:
df_dev.rename(columns={"discount_price__amount": "discount_price_amount", "price_detail__amount": "price_detail_amount"},inplace=True)

# Remove redundant columns
redundant_columns = ["discount_price__currency", "price_detail__currency","url"]
df_dev.drop(redundant_columns, inplace=True, axis=1)


# Convert time to year,month format

df_dev["published_time"] = df_dev["published_time"].apply(lambda x: x[:4]+'-'+x[5:7])

df_dev["created"] = df_dev["created"].apply(lambda x: x[:4]+'-'+x[5:7])

df_dev = df_dev.join(df_dev['published_time'].str.split('-', 1, expand=True).rename(columns={0:'published_time_year', 1:'published_time_month'}))

df_dev.drop(columns = "published_time",inplace=True)


# Convert NA discounts to 0

print(f"Number of courses with no discount = {df_dev['discount_price_amount'].isna().sum()}")

# Fill discount_price_amount = 0 for no discount
df_dev['discount_price_amount']=df_dev['discount_price_amount'].fillna(0)

print(f"Number of courses with no discount = {df_dev['discount_price_amount'].isna().sum()}")


# Drop avg_rating_recent,rating

df_dev.drop(columns = ["avg_rating_recent","rating"],inplace=True)
df_dev.head()

Number of courses with no discount = 168
Number of courses with no discount = 0


,id,title,is_paid,num_subscribers,avg_rating,num_reviews,num_published_lectures,num_published_practice_tests,created,discount_price_amount,price_detail_amount,published_time_year,published_time_month
0,567828,2020 Complete Python Bootcamp: From Zero to He...,True,1086954,4.56326,314098,152,0,2015-07,455.0,8640,2015,10
1,625204,The Web Developer Bootcamp,True,592474,4.64777,181621,405,0,2015-09,455.0,8640,2015,11
2,950390,Machine Learning A-Z™: Hands-On Python & R In ...,True,687330,4.55210,131007,323,0,2016-09,455.0,8640,2016,09
3,756150,Angular - The Complete Guide (2020 Edition),True,417746,4.57687,128705,455,0,2016-02,462.0,8640,2016,02
4,533682,Java Programming Masterclass for Software Deve...,True,497195,4.56571,122306,400,0,2015-06,455.0,8640,2015,07


#### df_it preprocessing

In [5]:
df_it = pd.read_csv("Dataset/IT_Software.csv")

In [6]:
df_it.head()

,id,title,url,is_paid,num_subscribers,avg_rating,avg_rating_recent,rating,num_reviews,is_wishlisted,num_published_lectures,num_published_practice_tests,created,published_time,discount_price__amount,discount_price__currency,discount_price__price_string,price_detail__amount,price_detail__currency,price_detail__price_string
0,762616,The Complete SQL Bootcamp 2020: Go from Zero t...,/course/the-complete-sql-bootcamp/,True,295509,4.66019,4.67874,4.67874,78006,False,84,0,2016-02-14T22:57:48Z,2016-04-06T05:16:11Z,455.0,INR,₹455,8640.0,INR,"₹8,640"
1,937678,Tableau 2020 A-Z: Hands-On Tableau Training fo...,/course/tableau10/,True,209070,4.58956,4.60015,4.60015,54581,False,78,0,2016-08-22T12:10:18Z,2016-08-23T16:59:49Z,455.0,INR,₹455,8640.0,INR,"₹8,640"
2,1361790,PMP Exam Prep Seminar - PMBOK Guide 6,/course/pmp-pmbok6-35-pdus/,True,155282,4.59491,4.59326,4.59326,52653,False,292,2,2017-09-26T16:32:48Z,2017-11-14T23:58:14Z,455.0,INR,₹455,8640.0,INR,"₹8,640"
3,648826,The Complete Financial Analyst Course 2020,/course/the-complete-financial-analyst-course/,True,245860,4.54407,4.53772,4.53772,46447,False,338,0,2015-10-23T13:34:35Z,2016-01-21T01:38:48Z,455.0,INR,₹455,8640.0,INR,"₹8,640"
4,637930,An Entire MBA in 1 Course:Award Winning Busine...,/course/an-entire-mba-in-1-courseaward-winning...,True,374836,4.47080,4.47173,4.47173,41630,False,83,0,2015-10-12T06:39:46Z,2016-01-11T21:39:33Z,455.0,INR,₹455,8640.0,INR,"₹8,640"


In [7]:
df_it.rename(columns={"discount_price__amount": "discount_price_amount", "price_detail__amount": "price_detail_amount"},inplace=True)

# Remove redundant columns
redundant_columns = ["discount_price__currency", "price_detail__currency","url", "is_wishlisted","discount_price__price_string","price_detail__price_string"]
df_it.drop(redundant_columns, inplace=True, axis=1)


# Drop avg_rating_recent,rating

df_it.drop(columns = ["avg_rating_recent","rating"],inplace=True)
df_it.head()


# Convert time to year,month format

df_it["published_time"] = df_it["published_time"].apply(lambda x: x[:4]+'-'+x[5:7])

df_it["created"] = df_it["created"].apply(lambda x: x[:4]+'-'+x[5:7])

df_it = df_it.join(df_it['published_time'].str.split('-', 1, expand=True).rename(columns={0:'published_time_year', 1:'published_time_month'}))

df_it.drop(columns = "published_time",inplace=True)


# Convert NA discounts to 0

print(f"Number of courses with no discount = {df_it['discount_price_amount'].isna().sum()}")

# Fill discount_price_amount = 0 for no discount
df_it['discount_price_amount']=df_it['discount_price_amount'].fillna(0)

print(f"Number of courses with no discount = {df_it['discount_price_amount'].isna().sum()}")

df_it.head()

Number of courses with no discount = 1829
Number of courses with no discount = 0


,id,title,is_paid,num_subscribers,avg_rating,num_reviews,num_published_lectures,num_published_practice_tests,created,discount_price_amount,price_detail_amount,published_time_year,published_time_month
0,762616,The Complete SQL Bootcamp 2020: Go from Zero t...,True,295509,4.66019,78006,84,0,2016-02,455.0,8640.0,2016,04
1,937678,Tableau 2020 A-Z: Hands-On Tableau Training fo...,True,209070,4.58956,54581,78,0,2016-08,455.0,8640.0,2016,08
2,1361790,PMP Exam Prep Seminar - PMBOK Guide 6,True,155282,4.59491,52653,292,2,2017-09,455.0,8640.0,2017,11
3,648826,The Complete Financial Analyst Course 2020,True,245860,4.54407,46447,338,0,2015-10,455.0,8640.0,2016,01
4,637930,An Entire MBA in 1 Course:Award Winning Busine...,True,374836,4.47080,41630,83,0,2015-10,455.0,8640.0,2016,01


In [8]:
df_dev.columns == df_it.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [9]:
print(f"{df_dev.shape[0]} + {df_it.shape[0]} = {df_dev.shape[0]+df_it.shape[0]}")

9932 + 22853 = 32785


In [10]:
frames = [df_dev,df_it]
df = pd.concat(frames)
print(df.shape[0])

df.drop_duplicates(inplace=True)
temp1 = df.shape[0]
print(temp1)

df = df.sample(frac=1).reset_index(drop=True)

32785
32764


In [11]:
# Price
temp2 = df['price_detail_amount'].isna().sum()
print(f"Number of courses with na price = {temp2}")
df.dropna(inplace=True)
print(f"{temp1} - {temp2} = {temp1-temp2}")
print(df.shape[0])

Number of courses with na price = 497
32764 - 497 = 32267
32267


#### Final Dataset 

In [12]:
df.shape

(32267, 13)

## Preparing Category Column

#### Creating List of all possible courses, technology, frameworks and so on!!

In [13]:
####### PART 1 #######
# Read

f = open('Dataset/mega_list.txt')
mega_list = f.read().splitlines()
f.close()

print(len(mega_list))
print(type(mega_list))


####### PART 2 #######

# label tag
# user-select disable

# span tag
# .filter--count--33UW8 -> display:none

temp = '''
'''

####### PART 3 #######

temp = temp.split("\n")

print(len(mega_list))
print(len(temp))
print(len(mega_list+temp))

r = []
for i in temp:
    if i == '':
        continue
    t = i.lower().strip()
    if t not in r and t not in mega_list:
        r.append(t)
r = list(set(r))

print(len(r))

combine = mega_list+r
mega_list = sorted(combine)

print(len(mega_list))


####### PART 4 #######

# Write
with open('Dataset/mega_list.txt', 'w') as f:
    for item in mega_list:
        f.write('%s\n' % item)     
        
        
####### PART 5 #######

# Load Mega List

f = open('Dataset/mega_list.txt')
mega_list = f.read().splitlines()
f.close()

print(len(mega_list))
print(type(mega_list))

828
<class 'list'>
828
2
830
0
828
828
<class 'list'>


In [14]:
def keywords_mega_list(text,mega_list):
    result = []
    words = text.lower().split()
    for word in words:
        if (word in mega_list) and (word not in result):
            result.append(word)
            
    return result

In [15]:
keywords_mega_list("Learn Python From Scratch With Lots of Examples and Projects",mega_list)

['python']

In [16]:
'''
count=0
titles = list(set(df["title"]))
print(f"Number of distinct cources = {len(titles)} (out of {len(df['title'])})")

FINAL_DICT = dict()

for title in titles:   
    output = keywords_mega_list(title,mega_list)
    
    if title not in FINAL_DICT:
        FINAL_DICT[title] = output
    
    if output == []:
        # print(f"{title} => {output}")
        count+=1
print(count)

df['category'] = df['title'].map(FINAL_DICT)

df.to_csv("Software.csv",index=False)
'''

'\ncount=0\ntitles = list(set(df["title"]))\nprint(f"Number of distinct cources = {len(titles)} (out of {len(df[\'title\'])})")\n\nFINAL_DICT = dict()\n\nfor title in titles:   \n    output = keywords_mega_list(title,mega_list)\n    \n    if title not in FINAL_DICT:\n        FINAL_DICT[title] = output\n    \n    if output == []:\n        # print(f"{title} => {output}")\n        count+=1\nprint(count)\n\ndf[\'category\'] = df[\'title\'].map(FINAL_DICT)\n\ndf.to_csv("Software.csv",index=False)\n'

## FINAL DATASET - Software.csv

In [17]:
df = pd.read_csv("Dataset/Software.csv")
df.sample(10)

,id,title,is_paid,num_subscribers,avg_rating,num_reviews,num_published_lectures,num_published_practice_tests,created,discount_price_amount,price_detail_amount,published_time_year,published_time_month,category
16359,45139,"Get to know the VIX Index (aka ""The Fear Index"")",True,1036,4.75000,130,10,0,2013-03,0.0,5120.0,2013,3,[]
6848,1974520,Trading con Fibonacci - Retrocesos y expansiones,True,42,3.80000,20,15,0,2018-10,455.0,8640.0,2018,10,[]
30962,2539266,AWS Elastic Beanstalk - Master Class|Hands On ...,True,4016,4.68518,82,103,0,2019-09,455.0,1280.0,2019,11,[]
206,1359988,Business Management: Motivating Workers,True,258,3.80000,27,17,0,2017-09,455.0,8640.0,2017,10,[]
3876,1434980,Advanced Windows Presentation Foundation (WPF)...,True,2018,4.09091,331,64,0,2017-11,455.0,8640.0,2017,11,[]
13204,1513620,Getting Started with PowerShell DSC,True,99,4.15000,11,20,0,2018-01,455.0,8000.0,2018,1,['powershell']
2004,2758996,Investing in gold will protect your family,True,665,3.00000,3,9,0,2020-01,455.0,5120.0,2020,1,[]
22982,2235142,Complete AWS Big Data and Analytics Certification,True,2551,3.40000,21,39,0,2019-02,455.0,1600.0,2019,3,[]
2408,276058,The Complete Public Speaking Certification Pro...,True,5016,4.53175,418,23,0,2014-08,455.0,8640.0,2014,8,[]
18842,3275954,Java Blocks Course For Programmer,True,11901,3.20755,53,7,0,2020-06,455.0,8640.0,2020,7,['java']


## Keyword Extraction using Spacy Module (Optional Code)

In [18]:
import spacy
nlp = spacy.load('en_core_web_sm')

from collections import Counter
from string import punctuation
import pandas as pd

In [19]:
df = pd.read_csv("Dataset/Development.csv")
df = df.sample(100)

In [20]:
titles = list(set(df["title"]))
print(f"Number of distinct cources = {len(titles)} (out of {len(df['title'])})")

Number of distinct cources = 100 (out of 100)


In [21]:
def keywords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
            
        if(token.pos_ in pos_tag and token.text not in result):
            result.append(token.text)
                
    return result

In [22]:
for title in titles:
    output = keywords(title)
    print(f"{title} => {output}")

Learn Java Step by Step and become an Expert => ['java', 'step', 'expert']
Deploying machine learning models with flask for beginners => ['machine', 'learning', 'models', 'flask', 'beginners']
Microsoft SQL Server for Beginners to expert => ['microsoft', 'sql', 'server', 'beginners']
BDD with Selenium Webdriver and Specflow using C# => ['bdd', 'selenium', 'webdriver', 'c']
PHP OOP: Object Oriented Programming for beginners + Project => ['php', 'oop', 'object', 'programming', 'beginners', 'project']
Vagrant Up! Comprehensive development system automation => ['vagrant', 'comprehensive', 'development', 'system', 'automation']
Elasticsearch in Action => ['elasticsearch', 'action']
Learn C++ in Less than 4 Hours - for Beginners  => ['c++', 'hours', 'beginners']
Python 3 Basics Course => ['python', 'basics', 'course']
PHP: Complete Registration and Login with Email Verification => ['php', 'complete', 'registration', 'login', 'email', 'verification']
The Python Bible™ | Everything You Need to

DATA SCIENCE with MACHINE LEARNING and DATA ANALYTICS => ['data', 'science', 'machine', 'learning', 'analytics']
Unity from Zero to Proficiency (intermediate) => ['unity', 'proficiency', 'intermediate']
AWS Certified Developer - Associate Tutorial - Step 1 => ['aws', 'developer', 'associate', 'tutorial', 'step']
WebVR - Virtual Reality with A-Frame from Scratch => ['virtual', 'reality', 'frame', 'scratch']
RabbitMQ & Java (Spring Boot) for System Integration => ['rabbitmq', 'java', 'boot', 'system', 'integration']
Unity VR: Oculus Quest Bowling Game in 30 Minutes => ['unity', 'vr', 'oculus', 'quest', 'bowling', 'game', 'minutes']
Python and Flask Full-Stack Web Development for beginners => ['python', 'flask', 'stack', 'web', 'development', 'beginners']
Learn C++ Programming By Making Games Volume 2 => ['c++', 'programming', 'games', 'volume']
Introduction to jQuery with Visual Studio 2017 Community => ['introduction', 'jquery', 'visual', 'studio', 'community']
Learning Path: Web Develo

## Final Course Recommender

## Steps

### 1) Input course

### 2) Get category and avg_rating of the input course

### 3) For the given course, compute
### a) Similar Courses Vector (ITEM_VECTOR)
### b) Predict Ratings of the input course

In [23]:
df = pd.read_csv("Dataset/Software.csv")
df = df[df["category"] != "[]"]
df = df.dropna()
df = df.reset_index(drop=True)

In [24]:
# Find Similarity

stop = stopwords.words('english')

WORD = re.compile(r'\w+')
stemmer = PorterStemmer()

def get_cosine(vec1, vec2):
    # print vec1, vec2
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    a = []
    for i in words:
        for ss in wn.synsets(i):
            a.extend(ss.lemma_names())
    for i in words:
        if i not in a:
            a.append(i)
    a = set(a)
    w = [stemmer.stem(i) for i in a if i not in stop]
    return Counter(w)

def get_similarity(a, b):
    a = text_to_vector(a.strip().lower())
    b = text_to_vector(b.strip().lower())

    return get_cosine(a, b)

def get_char_wise_similarity(a, b):
    a = text_to_vector(a.strip().lower())
    b = text_to_vector(b.strip().lower())
    s = []

    for i in a:
        for j in b:
            s.append(get_similarity(str(i), str(j)))
    try:
        return sum(s)/float(len(s))
    except: # len(s) == 0
        return 0

# get_similarity('Advanced Cyber & Social Media Intelligence Analysis', 'Python for Data Science and Machine Learning Bootcamp')

# Generate ITEM_VECTOR

def similar_vector(df_train,input_course,input_category):
    ITEM_VECTOR = [0 for i in range(df_train.shape[0])]
    cat = input_category
    count = 1
    print("\n\n\nComputing ITEM Vector")
    for q in range(df_train.shape[0]):
        print(count,end=" ",flush=True)
        count+=1
        sim = get_similarity(input_course, df_train.iloc[q]["title"])
        
        if cat:
            for j in cat:
                if j.strip().lower() in  df_train.iloc[q]["title"].strip().lower() and sim:
                    ITEM_VECTOR[q] = 1
    print("\n\n\n")
    return np.array(ITEM_VECTOR)

In [25]:
df_train = df.sample(2000).reset_index(drop=True)

In [26]:
print(f"{df.shape} {df_train.shape} Test = {(df.shape[0] - df_train.shape[0],df.shape[1])}")

(11707, 14) (2000, 14) Test = (9707, 14)


In [27]:
df_test = df[~df.id.isin(df_train.id)]
df_test.reset_index(drop=True,inplace=True)

In [28]:
list(df_test["title"])

['Javascript Programming For Everyone',
 'How to facilitate online Scrum and Agile training with Miro',
 'Python Scripting in Blender with Practical Projects',
 'Oracle PL/SQL is My Game: EXAM 1Z0-144',
 'Python Concurrency Simplified',
 'Structures, Unions, Enums for C Programming MCQ Practice',
 'SQL For Beginners: Learn to communicate with databases',
 'Complete Python 3 Masterclass Journey',
 'Learn SQL with Examples - Simple Select',
 'Cisco CCNP Route 300-101 Practice Tests',
 'HTML FOR BEGINNERS',
 'Blockchain for Business: How To Make Money With Blockchain',
 'Getting Started with .NET Core Clean Architecture',
 'Version Control with Git',
 'Agile Scrum Training, Scrum Master Certification Prep, Tests',
 'Automation Solutions with Chef Automate',
 'Build a Linux Based Raspberry Pi Drone',
 'Dart Masterclass Programming Course: iOS/Android Bible',
 'Laravel Tutorial for Beginners Step by Step',
 'PHP in Plain English',
 'Dart - Advanced Course',
 'Build e-commerce OOP website wi

In [29]:
# Generate similar items based on given title

input_course = input("Enter course title = ")

for i in range(df_test.shape[0]):
    if df_test.iloc[i]["title"] == input_course:
        print(df_test.iloc[i]["category"])
        input_category = ast.literal_eval(df_test.iloc[i]["category"])
        true_rating = df_test.iloc[i]["avg_rating"]
        break
        

similar_courses = []
ratings = []
ITEM_VECTOR = similar_vector(df_train,input_course,input_category)


print("Input Course = ",input_course)
for i in range(df_train.shape[0]):
    if ITEM_VECTOR[i]:
        similar_courses.append(df_train.iloc[i]["title"])
        ratings.append(df_train.iloc[i]["avg_rating"])

if similar_courses:
    print("\nNumber of similar courses = ",len(similar_courses))
    for i in similar_courses:
        print(i)
        
else:
    print("\nNo Similar Courses Found")
    
    
    
## Predict the rating of input_course using similar courses

if ratings:
    print("\n\n")
    average_rating_input_course = sum(ratings)/len(ratings)
    print("Predicted Ratings = ",round(average_rating_input_course,2))
    print("True Ratings = ",round(true_rating,2))

    error_percent = round((abs(average_rating_input_course - true_rating)/true_rating)*100,2)
    print(f"Error in predicted ratings = {error_percent} %")

Enter course title = Blockchain for Business: How To Make Money With Blockchain
['blockchain']



Computing ITEM Vector


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

1861 1862 1863 1864 1865 1866 1867 1868 1869 1870 1871 1872 1873 1874 1875 1876 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889 1890 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 



Input Course =  Blockchain for Business: How To Make Money With Blockchain

Number of similar courses =  50
The Complete Blockchain Professional Course
Blockchain & Cryptocurrency (Bitcoin, Ethereum) Essentials
Learn Blockchain By Building Your Own In JavaScript
Intro to Building Blockchain Apps